In [1]:
include("ct_utils.jl")
using .CTUtils
using DataFrames
using CSV 
using PyPlot

In [ ]:
df_ct = ct_to_df("../simulation_data/chargetransport/si-topcon-ill-sc.csv")
df_scaps = scaps_to_df("../simulation_data/scaps/si-topcon-ill-sc.eb")

In [ ]:
compare_densities(df_ct, df_scaps)
display(gcf())

In [ ]:
compare_bands(df_ct, df_scaps)
display(gcf())